# Test Data

In [ ]:
import os
import pandas as pd

prefix = '<path_to>/fov_segmentation'

df = pd.read_pickle(os.path.join(prefix, 'light_log.pkl'))
test_df = df[df.test == True]
test_df

# Benchmark Endoseg

In [ ]:
import time
import numpy as np
import endoseg

from utils import IoU

# load model with default values https://github.com/luiscarlosgph/endoseg/blob/25dc910836193f8cc5ccccaddd78e95aada46682/src/run.py#L45
segmenter = endoseg.Segmenter(
    min_hsv_thresh=[0, 0, 0], 
    max_hsv_thresh=[255, 255, 10], 
    deinterlace=False, denoise=True
)

# log df
logs_endoseg = pd.DataFrame(columns=['time', 'iou'])

for idx, row in test_df.iterrows():
    # load and prepare data
    img = np.load(os.path.join(prefix, row.image['folder'], row.image['file']))
    seg_gt = np.load(os.path.join(prefix, row.segmentation['folder'], row.segmentation['file']))

    # benchmark execution time
    start = time.time()
    seg = segmenter.segment(img)
    execution_time = time.time() - start
    
    # benchmark iou
    iou = IoU(seg, seg_gt)

    # store logs
    logs_endoseg = logs_endoseg.append(
        {
            'time': execution_time,
            'iou': iou
        }, ignore_index=True
    )

# Benchmark Deepseg

In [ ]:
import time
import numpy as np
from kornia import image_to_tensor, tensor_to_image
from lightning_modules import ImageSegmentationModule

from utils import IoU, load_yaml

device = 'cuda' # 'cpu' for cpu exection

# load model with default values
checkpoint_prefix = 'checkpoints/resnet_34'
checkpoint = 'epoch=35-step=1079.ckpt'

config = load_yaml(os.path.join(prefix, checkpoint_prefix, 'config.yml'))

model = ImageSegmentationModule(**config['model'])
model = model.load_from_checkpoint(os.path.join(prefix, checkpoint_prefix, checkpoint), **config['model'])
model = model.eval()
model.freeze()
model = model.to(device)

# log df
logs_deepseg = pd.DataFrame(columns=['time', 'iou'])

for idx, row in test_df.iterrows():
    # load and prepare data
    img = np.load(os.path.join(prefix, row.image['folder'], row.image['file']))
    seg_gt = np.load(os.path.join(prefix, row.segmentation['folder'], row.segmentation['file']))

    # benchmark execution time
    start = time.time()
    img = image_to_tensor(img, keepdim=False)
    img = img.to(model.device)
    img = img.float()/255.
    seg = model(img)
    execution_time = time.time() - start

    # benchmark iou
    seg = tensor_to_image(seg, keepdim=False)
    iou = IoU(seg, seg_gt)

    # store logs
    logs_deepseg = logs_deepseg.append(
        {
            'time': execution_time,
            'iou': iou
        }, ignore_index=True
    )

In [ ]:
print('Endoseg: Execution time {:.4f} +/- {:.4f}, Rate {:.1f}, IoU: {:.3f} +/- {:.3f}'.format(
        logs_endoseg.time.mean(), 
        logs_endoseg.time.std(), 
        1/logs_endoseg.time.mean(), 
        logs_endoseg.iou.mean(), 
        logs_endoseg.iou.std()
    )
)

print('Deepseg: Execution time {:.4f} +/- {:.4f}, Rate {:.1f}, IoU: {:.3f} +/- {:.3f}'.format(
        logs_deepseg.time.mean(), 
        logs_deepseg.time.std(), 
        1/logs_deepseg.time.mean(), 
        logs_deepseg.iou.mean(), 
        logs_deepseg.iou.std()
    )
)

# Export Deepseg

In [ ]:
import torch
from lightning_modules import ImageSegmentationModule

from utils import load_yaml

device = 'cuda' # 'cpu' for cpu exection

# load model with default values
checkpoint_prefix = 'checkpoints/resnet_34'
checkpoint = 'epoch=35-step=1079.ckpt'

config = load_yaml(os.path.join(prefix, checkpoint_prefix, 'config.yml'))

model = ImageSegmentationModule(**config['model'])
model = model.load_from_checkpoint(os.path.join(prefix, checkpoint_prefix, checkpoint), **config['model'])

script = model.to_torchscript(method='trace')
torch.jit.save(script, 'module.pt')